# How Many Agencies Does Covr Have Access To?

## Purpose
This notebook estimates the number of agencies that Covr could potentially connect with through its nursing facility clients.

## Required External Files
1. `covr_addresses.csv`
2. `all_nursing_home_coordinates.csv`

**Note:** These files may be updated as needed.

## Coordinate Generation
If `covr_coordinates.csv` doesn't exist in the parent folder:
- You will be prompted for a Google Maps API key
- The file will be automatically generated for you

**Important:** If you update `covr_addresses.csv`, you must:
1. Delete `covr_coordinates.csv`
2. Run the notebook again to regenerate coordinates for your new address list


# Imports

In [ ]:
import numpy as np
import random
import tqdm
from scipy.stats import norm
from tqdm import trange, tqdm
import matplotlib.pyplot as plt
import pandas as pd
from geopy.distance import geodesic
import math
from scipy.stats import gaussian_kde
from scipy.spatial import cKDTree
import multiprocessing as mp

import seaborn as sns
from scipy import stats
from google.colab import drive
import os
import requests
import time
from collections import Counter
from functools import partial
import pandas as pd
from matplotlib.lines import Line2D
try:
  from mpl_toolkits.basemap import Basemap
except:
  !pip install basemap
  from mpl_toolkits.basemap import Basemap


# Parameters



In [ ]:
### PARAMETERS ###

# MARKET ASSUMPTIONS
num_agencies = 8000 # number of agencies in the market
mean_agencies = 3 # mean number of agencies per facility
std_dev_agencies = 1 # standard deviation of agencies per facility
proportion_covr_facilities_use_agency = .75 # how many of covr's facilities use agency?

# FACILITY BEHAVIOR ASSUMPTIONS
distance_cutoff = 200  # Miles beyond which a facility will never connect with an agency
distance_midpoint = 70  # Distance (in miles) at which probability of a facility considering an agency is 0.5
steepness = 0.3  # Controls how quickly probability of connecting with an agency drops off with distance

# CONFIDENCE SIMULATION HELPER VALUES
confidence = .95  # confidence interval
samples = 100  # number of simulations to perform

In [ ]:
# Google Drive Setup
drive.mount('/drive', force_remount=True)
directory = '/drive/My Drive/Covr Agency Simulator/'

covr_addresses_path = os.path.join(directory, 'covr_addresses.csv')
covr_coordinates_path = os.path.join(directory, 'covr_coordinates.csv')
all_nursing_home_coordinates = pd.read_csv(os.path.join(directory, 'all_nursing_home_coordinates.csv'))

# Generate New Covr Coordinates

In [ ]:
def prompt_for_api_key():
    return input("Please enter your Google Maps API key: ")

def address_to_geocode(address, api_key):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address, 'key': api_key}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Geocoding failed for address: {address}. Status: {data['status']}")
            if 'error_message' in data:
                print(f"Error message: {data['error_message']}")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Request failed for address: {address}. Error: {e}")
        return None, None
    # finally:
        # time.sleep(0.1)

def process_csv(input_file, api_key):
    df = pd.read_csv(input_file)
    address_column = df.columns[0]

    geocoded_data = []
    skipped_addresses = []

    def geocode_with_retry(address, max_retries=3):
        for attempt in range(max_retries):
            lat, lng = address_to_geocode(address, api_key)
            if lat is not None and lng is not None:
                return lat, lng
            time.sleep(1)
        return None, None

    for index, row in df.iterrows():
        address = row[address_column]
        lat, lng = geocode_with_retry(address)
        if lat is not None and lng is not None:
            geocoded_data.append({'Address': address, 'Latitude': lat, 'Longitude': lng})
        else:
            skipped_addresses.append(address)

    # Create a new DataFrame with only successfully geocoded addresses
    result_df = pd.DataFrame(geocoded_data)

    # Log skipped addresses
    if skipped_addresses:
        print("The following addresses were skipped:")
        for addr in skipped_addresses:
            print(addr)

        # Optionally, save skipped addresses to a file
        with open('skipped_addresses.txt', 'w') as f:
            f.write('\n'.join(skipped_addresses))
        print("Skipped addresses have been saved to 'skipped_addresses.txt'")

    return result_df

# Check if the geocoded file already exists
if os.path.exists(covr_coordinates_path):
    print(f"The file {covr_coordinates_path} already exists.")
    covr_coordinates = pd.read_csv(covr_coordinates_path)
    display(covr_coordinates)
else:
    print(f"The file {covr_coordinates_path} does not exist. Processing addresses...")

    # Ensure the input file exists
    if not os.path.exists(covr_addresses_path):
        raise FileNotFoundError(f"The input file {covr_addresses_path} does not exist.")

    # Prompt for API key
    api_key = prompt_for_api_key()

    # Run the csv through the google maps API and return a DF
    covr_coordinates = process_csv(covr_addresses_path, api_key)
    display(covr_coordinates)

    # Save the result to the output CSV file
    covr_coordinates.to_csv(covr_coordinates_path, index=False)
    print(f"Results saved to {covr_coordinates_path}")

# Run Simulation

In [ ]:
# This function generates a dataframe of simulated agencies based on the distribution of the nursing facility market in all_nursing_home_coordinates

class Agency:
    def __init__(self, id, latitude, longitude):
        self.location = (latitude, longitude)
        self.id = id

def simulate_agency_coordinates(all_nursing_home_coordinates, covr_coordinates, num_agencies, max_offset_miles):
    # Extract coordinates
    coords = all_nursing_home_coordinates[['LONGITUDE', 'LATITUDE']].values

    # Randomly sample with replacement
    sampled_indices = np.random.choice(len(coords), num_agencies, replace=True)
    sampled_points = coords[sampled_indices]

    # Add small random offset
    offset_lat = np.random.uniform(-max_offset_miles/69, max_offset_miles/69, num_agencies)
    offset_lon = np.random.uniform(-max_offset_miles/54.6, max_offset_miles/54.6, num_agencies)

    simulated_points = sampled_points + np.column_stack((offset_lon, offset_lat))

    # Create a DataFrame with the simulated coordinates
    simulated_agencies = pd.DataFrame({
        'Longitude': simulated_points[:, 0],
        'Latitude': simulated_points[:, 1]
    })

    # Visualize the results
    plt.figure(figsize=(10, 7))

    # Create the Basemap
    m = Basemap(llcrnrlon=-125, llcrnrlat=23, urcrnrlon=-65, urcrnrlat=50,
                projection='lcc', lat_1=33, lat_2=45, lon_0=-95, resolution='l')

    # Draw map features
    m.drawcoastlines(linewidth=0.5)
    m.drawcountries(linewidth=0.5)
    m.drawstates(linewidth=0.3)
    m.fillcontinents(color='#FFEECC', lake_color='#99B3CC', alpha=0.3)
    m.drawmapboundary(fill_color='#99B3CC')

    # Convert lat/lon to map coordinates
    x, y = m(coords[:, 0], coords[:, 1])
    sim_x, sim_y = m(simulated_agencies['Longitude'], simulated_agencies['Latitude'])
    covr_x, covr_y = m(covr_coordinates['Longitude'], covr_coordinates['Latitude'])

    # Plot the data
    m.scatter(x, y, alpha=0.4, s=.5, color='blue', zorder=3)
    m.scatter(sim_x, sim_y, alpha=0.4, s=.5, color='green', zorder=4)
    m.scatter(covr_x, covr_y, alpha=1, s=2, color='red', zorder=5)

    # Create custom legend elements
    legend_elements = [
      Line2D([0], [0], marker='o', color='w', label='All Facilities',
            markerfacecolor='blue', markersize=10),
      Line2D([0], [0], marker='o', color='w', label='Simulated Agencies',
            markerfacecolor='green', markersize=10),
      Line2D([0], [0], marker='o', color='w', label='Covr facilities',
            markerfacecolor='red', markersize=10)
    ]

    # Add the legend with custom elements
    plt.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=3)

    plt.title(f'Distribution of Nursing Homes and Simulated Agencies (Max Offset: {max_offset_miles} miles)', fontsize=16)
    plt.tight_layout()
    plt.show()

    return simulated_agencies

In [ ]:
def calculate_distance(coord1, coord2):
    """Calculate distance between two coordinates in miles."""
    return geodesic(coord1, coord2).miles

def calculate_weight(distance, midpoint=distance_midpoint, steepness=steepness):
    """Calculate how reasonable an agency is based on distance"""
    # This calculation is a simple logistic function graphed below
    return 1 / (1 + math.exp((distance - midpoint) / (midpoint * steepness)))

def plot_distance_weight_curve(midpoint=distance_midpoint, steepness=steepness, max_distance=distance_cutoff):
    distances = np.linspace(0, max_distance, 1000)
    weights = [calculate_weight(d, midpoint, steepness) for d in distances]

    plt.figure(figsize=(8, 5))
    plt.plot(distances, weights)
    plt.title(f'Agency Distance Dropoff Curve (Midpoint: {midpoint}, Steepness: {steepness})')
    plt.xlabel('Distance (miles)')
    plt.ylabel('Weight')
    plt.grid(True)
    plt.show()

In [ ]:
class Facility:
    def __init__(self, latitude, longitude):
        self.location = (latitude, longitude)
        self.connected_agencies = []
        self.reasonable_agencies = []

    def connect_with_agency(self, agency):
        if not isinstance(agency, Agency):
            raise ValueError("Agency must be of type Agency")

        # Check if the agency is already connected
        if any(connected_agency.id == agency.id for connected_agency in self.connected_agencies):
            return False

        # If it's not already connected, add it to the list
        self.connected_agencies.append(agency)
        return True

    def add_reasonable_agencies(self, all_agencies):
      # We go through the list of all the agencies we generated
      for agency in all_agencies:
        # And if they are within our geography we add them to our list of reasonable agencies to choose from later
        distance = calculate_distance(self.location, agency.location)
        if distance <= distance_cutoff:
          self.reasonable_agencies.append(agency)

    def select_weighted_agency(self):
        if not self.reasonable_agencies:
            return None

        # Filter out already connected agencies
        available_agencies = [(agency, weight) for agency, weight in self.reasonable_agencies
                              if agency not in self.connected_agencies]

        if not available_agencies:
            return None

        agencies, weights = zip(*available_agencies)
        total_weight = sum(weights)
        normalized_weights = [w / total_weight for w in weights]

        selected_agency = random.choices(agencies, weights=normalized_weights, k=1)[0]
        return selected_agency

    def remove_all_connected_agencies(self):
        self.connected_agencies = []

    def get_all_agencies(self):
        return self.connected_agencies

    def get_connected_agencies(self):
        return self.connected_agencies
    def get_reasonable_agencies(self):
        return self.reasonable_agencies

In [ ]:
def process_facility(args):
    facility, tree, all_agencies, distance_cutoff, distance_midpoint, steepness = args
    distances, indices = tree.query(facility.location, k=len(all_agencies), distance_upper_bound=distance_cutoff)

    facility.reasonable_agencies = []
    for dist, idx in zip(distances, indices):
        if idx < len(all_agencies):
            agency = all_agencies[idx]
            exact_distance = geodesic(facility.location, agency.location).miles
            if exact_distance <= distance_cutoff:
                weight = calculate_weight(exact_distance, distance_midpoint, steepness)
                facility.reasonable_agencies.append((agency, weight))

    return facility

def optimize_facility_agency_matching(covr_coordinates, agencies, distance_cutoff=distance_cutoff, distance_midpoint=distance_midpoint, steepness=steepness, proportion_covr_facilities_use_agency=proportion_covr_facilities_use_agency):
    print("Creating facilities...")
    all_facilities = [Facility(row['Latitude'], row['Longitude']) for _, row in tqdm(covr_coordinates.iterrows(), total=len(covr_coordinates))]

    # Determine which facilities use agencies
    num_facilities_use_agency = int(len(all_facilities) * proportion_covr_facilities_use_agency)
    facilities_use_agency = np.random.choice(all_facilities, num_facilities_use_agency, replace=False)

    print(f"Number of facilities that use agencies: {num_facilities_use_agency}")

    print("Building KD-Tree for agencies...")
    agency_coords = np.array([[agency.location[0], agency.location[1]] for agency in tqdm(agencies)])
    tree = cKDTree(agency_coords)

    print("Matching facilities with agencies...")
    with mp.Pool(mp.cpu_count()) as pool:
        args = [(facility, tree, agencies, distance_cutoff, distance_midpoint, steepness) for facility in facilities_use_agency]
        facilities_with_agencies = list(tqdm(pool.imap(process_facility, args), total=len(facilities_use_agency)))

    print(f"Processed {len(facilities_with_agencies)} facilities that use agencies.")
    avg_num_reasonable_agencies = sum(len(f.get_reasonable_agencies()) for f in facilities_with_agencies) / len(facilities_with_agencies)
    print(f"Average number of reasonable agencies per facility (for facilities that use agencies): {avg_num_reasonable_agencies:.2f}")

    return facilities_with_agencies

def run_facility_agency_connection(facilities, mean_agencies=mean_agencies, std_dev_agencies=std_dev_agencies):
    total_connections = 0
    all_connected_agencies = set()  # We'll use this to track all unique connected agencies

    for facility in facilities:
        facility.remove_all_connected_agencies()
        num_agencies = max(0, int(np.random.normal(mean_agencies, std_dev_agencies)))
        facility_connections = 0
        for _ in range(num_agencies):
            selected_agency = facility.select_weighted_agency()
            if selected_agency:
                if facility.connect_with_agency(selected_agency):
                    facility_connections += 1
            else:
                break
        total_connections += facility_connections

        # After processing each facility, add its connected agencies to the overall set
        all_connected_agencies.update(agency.id for agency in facility.get_connected_agencies())

    # Now we have the correct count of unique agencies
    unique_connected_agencies = len(all_connected_agencies)

    return unique_connected_agencies, total_connections / len(facilities)

def estimate_unique_agencies_with_confidence(facilities, mean_agencies=mean_agencies, std_dev_agencies=std_dev_agencies, confidence=confidence, n_iterations=samples):
    unique_agency_counts = []
    avg_agencies_per_facility_counts = []

    for _ in range(n_iterations):
        # This line was already correct, but our interpretation of it was wrong
        unique_count, avg_count = run_facility_agency_connection(facilities, mean_agencies, std_dev_agencies)
        unique_agency_counts.append(unique_count)
        avg_agencies_per_facility_counts.append(avg_count)

    unique_agency_counts = np.array(unique_agency_counts)
    avg_agencies_per_facility_counts = np.array(avg_agencies_per_facility_counts)

    # Calculate confidence intervals
    ci_unique = stats.t.interval(confidence, len(unique_agency_counts) - 1,
                                 loc=np.mean(unique_agency_counts),
                                 scale=stats.sem(unique_agency_counts))

    ci_avg = stats.t.interval(confidence, len(avg_agencies_per_facility_counts) - 1,
                              loc=np.mean(avg_agencies_per_facility_counts),
                              scale=stats.sem(avg_agencies_per_facility_counts))

    # The fix is in correctly interpreting these results:
    print(f"Confidence interval is {confidence * 100}%")
    print(f"Number of unique agencies Covr is connected to:")
    print(f"  Min: {round(ci_unique[0])}")
    print(f"  Max: {round(ci_unique[1])}")
    print(f"  Mean: {round(np.mean(unique_agency_counts))}")
    print(f"\nAverage number of agencies per facility:")
    print(f"  Min: {ci_avg[0]:.2f}")
    print(f"  Max: {ci_avg[1]:.2f}")
    print(f"  Mean: {np.mean(avg_agencies_per_facility_counts):.2f}")

In [ ]:
### MAIN

# Create facilities
all_facilities = []
for index, row in covr_coordinates.iterrows():
    facility = Facility(row['Latitude'], row['Longitude'])
    facility.id = index
    all_facilities.append(facility)

# Create a list of agencies
agencies = []
simulated_agency_coordinates = simulate_agency_coordinates(all_nursing_home_coordinates, covr_coordinates, num_agencies=num_agencies, max_offset_miles=15)
print(f"Number of agencies generated: {len(simulated_agency_coordinates)}")
for index, row in simulated_agency_coordinates.iterrows():
    agency = Agency(index, row['Latitude'], row['Longitude'])
    agencies.append(agency)

# Simulate facility-agency connections
facilities_that_use_agency = []
facilities_that_use_agency = optimize_facility_agency_matching(covr_coordinates, agencies)
unique_agencies, avg_connections = run_facility_agency_connection(facilities_that_use_agency)
print(f"Number of unique agencies Covr is connected to: {unique_agencies}")
print(f"Average number of agencies per facility: {avg_connections:.2f}")
estimate_unique_agencies_with_confidence(facilities_that_use_agency)

In [ ]:
estimate_unique_agencies_with_confidence(facilities_that_use_agency)

# Plot Results

In [ ]:
# Plots

def plot_facility_agency_distribution(all_facilities, facilities_using_agencies):
    # Create a dictionary of facilities using agencies for faster lookup
    facilities_using_agencies_dict = {f.location: f for f in facilities_using_agencies}

    # Create a list of agency counts for all facilities
    agency_counts = [len(facilities_using_agencies_dict[f.location].connected_agencies)
                     if f.location in facilities_using_agencies_dict else 0
                     for f in all_facilities]

    # Count the occurrences of each number of connections
    count_distribution = Counter(agency_counts)

    # Determine the maximum number of connections
    max_connections = max(count_distribution.keys())

    # Prepare data for plotting
    x = range(max_connections + 1)  # 0 to max number of connections
    y = [count_distribution[i] for i in x]

    # Create the bar plot
    plt.figure(figsize=(12, 6))
    bars = plt.bar(x, y)
    plt.xlabel('Number of Agency Connections')
    plt.ylabel('Number of Facilities')
    plt.title('Distribution of Facilities by Number of Agency Connections')

    # Color bars based on agency usage
    for i, bar in enumerate(bars):
        if i == 0:
            bar.set_color('lightgray')
        else:
            bar.set_color('blue')

    # Add value labels on top of each bar
    for i, v in enumerate(y):
        if v > 0:
            plt.text(i, v, str(v), ha='center', va='bottom')

    # Set x-axis to show only integer values
    plt.xticks(x)

    # Add a legend
    plt.legend(['No Agencies', 'Uses Agencies'])

    plt.tight_layout()
    plt.show()

    # Print summary statistics
    facilities_using_agencies_count = sum(1 for count in agency_counts if count > 0)
    print(f"Total facilities: {len(all_facilities)}")
    print(f"Facilities using agencies: {facilities_using_agencies_count} ({facilities_using_agencies_count/len(all_facilities)*100:.2f}%)")
    print(f"Facilities not using agency: {len(all_facilities) - facilities_using_agencies_count}")
    if facilities_using_agencies_count > 0:
        print(f"Average agencies per facility (among those using agencies): {np.mean([count for count in agency_counts if count > 0]):.2f}")
    else:
        print("No facilities are using agencies.")

def plot_agency_coverage(facilities, n_sample=5):
    """Plot the coverage area of a sample of facilities with agency weights determining opacity."""
    plt.figure(figsize=(15, 10))
    m = Basemap(llcrnrlon=-125, llcrnrlat=23, urcrnrlon=-65, urcrnrlat=50,
                projection='lcc', lat_1=33, lat_2=45, lon_0=-95, resolution='l')

    m.drawcoastlines(linewidth=0.5)
    m.drawcountries(linewidth=0.5)
    m.drawstates(linewidth=0.3)
    m.fillcontinents(color='#FFEECC', lake_color='#99B3CC', alpha=0.3)
    m.drawmapboundary(fill_color='#99B3CC')

    colors = plt.cm.rainbow(np.linspace(0, 1, n_sample))

    for facility, color in zip(np.random.choice(facilities, n_sample, replace=False), colors):
        fac_x, fac_y = m(facility.location[1], facility.location[0])
        m.scatter(fac_x, fac_y, color=color, s=100, label=f'Facility at {facility.location}', zorder=5)

        # Normalize weights for this facility
        weights = [weight for _, weight in facility.reasonable_agencies]
        max_weight = max(weights)
        min_weight = min(weights)
        weight_range = max_weight - min_weight

        for agency, weight in facility.reasonable_agencies:
            agency_x, agency_y = m(agency.location[1], agency.location[0])

            # Normalize the weight to determine opacity
            if weight_range > 0:
                opacity = 0.1 + 0.8 * (weight - min_weight) / weight_range
            else:
                opacity = 0.5  # Default opacity if all weights are the same

            m.scatter(agency_x, agency_y, color=color, alpha=opacity, s=20, zorder=4)
            m.plot([fac_x, agency_x], [fac_y, agency_y], color=color, alpha=opacity*0.5, linewidth=1, zorder=3)

    plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.1), ncol=3)
    plt.title('A Few Facilities and Their Reasonable Agencies', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_facility_agency_distribution(all_facilities, facilities_that_use_agency)
plot_distance_weight_curve()
plot_agency_coverage(facilities_that_use_agency)